In [ ]:
C:\Users\coldbrew\VTON-project\data\dataset\image

In [2]:

import os, sys

notebook_dir = os.getcwd()
project_root_dir = os.path.abspath(os.path.join(notebook_dir, ".."))
dataset_person_folder = os.path.join(project_root_dir, "data", "dataset", "image")
dataset_person_upper_mask_folder = os.path.join(project_root_dir, "data", "dataset", "image_mask_U")
dataset_person_lower_mask_folder = os.path.join(project_root_dir, "data", "dataset", "image_mask_L")

dst_person_with_upper_agnostic_folder = os.path.join(project_root_dir, "data", "dataset", "image_with_upper_agnostic")
dst_person_with_lower_agnostic_folder = os.path.join(project_root_dir, "data", "dataset", "image_with_lower_agnostic")

# 결과 저장 폴더가 없으면 생성
os.makedirs(dst_person_with_upper_agnostic_folder, exist_ok=True)
os.makedirs(dst_person_with_lower_agnostic_folder, exist_ok=True)

# 사람이미지에 각각 마스크 이미지 넣어서 저장
import cv2
import numpy as np
from tqdm import tqdm

# 사람 이미지 파일 목록 가져오기 (마스크 이미지와 파일명이 동일하다고 가정)
person_files = sorted(os.listdir(dataset_person_folder))

for file_name in tqdm(person_files, desc="Processing images"):
    # 파일 경로 구성
    person_img_path = os.path.join(dataset_person_folder, file_name)
    upper_mask_path = os.path.join(dataset_person_upper_mask_folder, file_name)
    lower_mask_path = os.path.join(dataset_person_lower_mask_folder, file_name)
    
    # 이미지 로드 (마스크는 그레이스케일로 로드)
    person_img = cv2.imread(person_img_path)
    upper_mask = cv2.imread(upper_mask_path, cv2.IMREAD_GRAYSCALE)
    lower_mask = cv2.imread(lower_mask_path, cv2.IMREAD_GRAYSCALE)
    
    if person_img is None:
        print(f"Error loading person image: {file_name}")
        continue
    if upper_mask is None:
        print(f"Error loading upper mask image: {file_name}")
        continue
    if lower_mask is None:
        print(f"Error loading lower mask image: {file_name}")
        continue

    # 상체 마스크 적용: 마스크 영역을 검정색(0)으로 채우기
    upper_agnostic_img = person_img.copy()
    # binary 마스크라고 가정 (0: 배경, 255: 마스크 영역). 임계값 127을 기준으로 처리합니다.
    upper_agnostic_img[upper_mask > 127] = 0

    # 하체 마스크 적용: 마스크 영역을 검정색(0)으로 채우기
    lower_agnostic_img = person_img.copy()
    lower_agnostic_img[lower_mask > 127] = 0

    # 결과 저장
    cv2.imwrite(os.path.join(dst_person_with_upper_agnostic_folder, file_name), upper_agnostic_img)
    cv2.imwrite(os.path.join(dst_person_with_lower_agnostic_folder, file_name), lower_agnostic_img)

Processing images: 100%|██████████| 195/195 [00:02<00:00, 93.67it/s]
